In [112]:
import pandas as pd 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import RepeatedStratifiedKFold,cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, auc,roc_curve
from sklearn.preprocessing import StandardScaler
import numpy as np 

In [31]:
df_dataset_1 = pd.read_csv("project3_dataset1.txt",sep="\t", header=None)

df_dataset_2 = pd.read_csv("project3_dataset2.txt",sep="\t", header=None)


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.1166,0.19220,0.32150,0.16280,0.2572,0.06637,1
1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.52490,0.53550,0.17410,0.3985,0.12440,1
2,11.26,19.96,73.72,394.1,0.08020,0.11810,0.09274,0.05588,0.2595,0.06233,...,22.33,78.27,437.6,0.1028,0.18430,0.15460,0.09314,0.2955,0.07009,0
3,11.43,15.39,73.06,399.8,0.09639,0.06889,0.03503,0.02875,0.1734,0.05865,...,22.02,79.93,462.0,0.1190,0.16480,0.13990,0.08476,0.2676,0.06765,0
4,14.61,15.69,92.68,664.9,0.07618,0.03515,0.01447,0.01877,0.1632,0.05255,...,21.75,103.70,840.8,0.1011,0.07087,0.04746,0.05813,0.2530,0.05695,0


In [34]:
#Dataset 1
X,y = df_dataset_1.iloc[:,:-1], df_dataset_1.iloc[:,-1]
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [89]:
solvers = ['liblinear']
penalty = ['l2']
c_values = [1000, 100, 10, 5, 1.0, 0.1, 0.01]
clf = LogisticRegression()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=clf, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.978744 using {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.951626 (0.030284) with: {'C': 1000, 'penalty': 'l2', 'solver': 'liblinear'}
0.958277 (0.030203) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.964863 (0.028710) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.965588 (0.029307) with: {'C': 5, 'penalty': 'l2', 'solver': 'liblinear'}
0.978019 (0.021205) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.978744 (0.022382) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.969936 (0.024482) with: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}


In [94]:
kf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
score = cross_val_score(LogisticRegression(solver='liblinear', penalty='l2',C=9), X_train, y_train, cv= kf, scoring="accuracy")
score.mean()
#clf = LogisticRegression(solver='liblinear', penalty='l2',C=0.1)
#clf.fit(X_train,y_train)
#clf.score(X_test,y_test)
#y_pred = clf.predict(X_test)
#clf.get_params()
#print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(clf.score(X_train, y_train)))
#print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))
#print('Precision of logistic regression classifier on test set: {:.2f}'.format(precision_score(y_test, y_pred)))

0.9655877616747183

In [242]:
from sklearn.preprocessing import LabelEncoder
 #Dataset 1
X,y = df_dataset_2.iloc[:,:-1], df_dataset_2.iloc[:,-1]
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.head()
# Creating a instance of label Encoder.
le = LabelEncoder()
kf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scaler = StandardScaler()
X_train_without_categorical = X_train.iloc[:,X_train.columns!=4]
#X_train_categorical = pd.get_dummies(X_train.iloc[:,X_train.columns==4]).to_numpy()
X_train_with_categorical = le.fit_transform(X_train.iloc[:,X_train.columns==4]).reshape(-1,1)
X_train = np.append(X_train_without_categorical,X_train_with_categorical,1)
X_train = scaler.fit_transform(X_train)
X_train.shape
#score = cross_val_score(LogisticRegression(solver='liblinear', penalty='l2',C=9), X_train, y_train, cv= kf, scoring="accuracy")
#score.mean()

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(369, 9)

In [248]:
solvers = ['liblinear']
penalty = ['l2']
c_values = np.linspace(0.1,10,50)
print(c_values)
clf = LogisticRegression()
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=clf, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)
#print(grid_result)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

[ 0.1         0.30204082  0.50408163  0.70612245  0.90816327  1.11020408
  1.3122449   1.51428571  1.71632653  1.91836735  2.12040816  2.32244898
  2.5244898   2.72653061  2.92857143  3.13061224  3.33265306  3.53469388
  3.73673469  3.93877551  4.14081633  4.34285714  4.54489796  4.74693878
  4.94897959  5.15102041  5.35306122  5.55510204  5.75714286  5.95918367
  6.16122449  6.36326531  6.56530612  6.76734694  6.96938776  7.17142857
  7.37346939  7.5755102   7.77755102  7.97959184  8.18163265  8.38367347
  8.58571429  8.7877551   8.98979592  9.19183673  9.39387755  9.59591837
  9.79795918 10.        ]
Best: 0.726476 using {'C': 0.9081632653061225, 'penalty': 'l2', 'solver': 'liblinear'}
0.722823 (0.067090) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.726451 (0.065127) with: {'C': 0.3020408163265306, 'penalty': 'l2', 'solver': 'liblinear'}
0.725551 (0.064134) with: {'C': 0.5040816326530613, 'penalty': 'l2', 'solver': 'liblinear'}
0.725551 (0.064134) with: {'C': 0.70612244